## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Qaanaaq,GL,2021-10-30 20:22:02,77.4840,-69.3632,2.16,53,0,8.72
1,1,Vaini,TO,2021-10-30 20:18:40,-21.2000,-175.2000,77.16,88,20,9.22
2,2,Quatre Cocos,MU,2021-10-30 20:22:03,-20.2078,57.7625,71.71,77,46,7.81
3,3,Bluff,NZ,2021-10-30 20:22:03,-46.6000,168.3333,54.91,66,89,20.27
4,4,Ushuaia,AR,2021-10-30 20:22:03,-54.8000,-68.3000,51.46,50,0,19.57


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimun temperature you would like for your trip? "))
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

What is the minimun temperature you would like for your trip? 75
What is the maximun temperature you would like for your trip? 78


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Vaini,TO,2021-10-30 20:18:40,-21.2000,-175.2000,77.16,88,20,9.22
12,12,Cayhagan,PH,2021-10-30 20:22:06,9.6697,122.4256,77.85,85,100,11.01
37,37,Galle,LK,2021-10-30 20:22:13,6.0367,80.2170,77.25,89,77,3.11
38,38,Rikitea,PF,2021-10-30 20:22:14,-23.1203,-134.9692,75.00,85,100,21.14
43,43,Polewali,ID,2021-10-30 20:22:15,-3.4324,119.3435,75.90,94,100,2.53


In [5]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count()

66


City_ID       66
City          66
Country       66
Date          66
Lat           66
Lng           66
Max Temp      66
Humidity      66
Cloudiness    66
Wind Speed    66
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
print(len(clean_df))
clean_df.head()

66


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Vaini,TO,2021-10-30 20:18:40,-21.2000,-175.2000,77.16,88,20,9.22
12,12,Cayhagan,PH,2021-10-30 20:22:06,9.6697,122.4256,77.85,85,100,11.01
37,37,Galle,LK,2021-10-30 20:22:13,6.0367,80.2170,77.25,89,77,3.11
38,38,Rikitea,PF,2021-10-30 20:22:14,-23.1203,-134.9692,75.00,85,100,21.14
43,43,Polewali,ID,2021-10-30 20:22:15,-3.4324,119.3435,75.90,94,100,2.53


In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Vaini,TO,77.16,-21.2000,-175.2000,
12,Cayhagan,PH,77.85,9.6697,122.4256,
37,Galle,LK,77.25,6.0367,80.2170,
38,Rikitea,PF,75.00,-23.1203,-134.9692,
43,Polewali,ID,75.90,-3.4324,119.3435,
70,Ambilobe,MG,75.96,-13.2000,49.0500,
72,Avarua,CK,75.25,-21.2078,-159.7750,
87,Atuona,PF,77.85,-9.8000,-139.0333,
100,Wamba,NG,76.33,8.9333,8.6000,
102,Iguatemi,BR,76.69,-23.6803,-54.5611,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Vaini,TO,77.16,-21.2000,-175.2000,Keleti Beach Resort
12,Cayhagan,PH,77.85,9.6697,122.4256,Melrose Beach Resort
37,Galle,LK,77.25,6.0367,80.2170,Amangalla
38,Rikitea,PF,75.00,-23.1203,-134.9692,People ThankYou
43,Polewali,ID,75.90,-3.4324,119.3435,Lilianto Hotel
70,Ambilobe,MG,75.96,-13.2000,49.0500,Hôtel & Restaurant DIANA Ambilobe
72,Avarua,CK,75.25,-21.2078,-159.7750,Paradise Inn
87,Atuona,PF,77.85,-9.8000,-139.0333,Villa Enata
100,Wamba,NG,76.33,8.9333,8.6000,Roma Zee Guest House Annex
102,Iguatemi,BR,76.69,-23.6803,-54.5611,Hotel Seriema


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))